In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import math
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

In [3]:
# Product catalog
PRODUCTS = {'Truck': {'models': [('Cascadia DD15', 'Class 8 Truck', 'Sleeper Cab, Auto Trans, 455HP', 145000), 
                                 ('Cascadia DD13', 'Class 8 Truck', 'Day Cab, Manual Trans, 410HP', 135000), 
                                 ('LT Series', 'Class 7 Truck', 'Extended Cab, Auto Trans, 360HP', 125000), 
                                 ('M2 106', 'Class 6 Truck', 'Crew Cab, Auto Trans, 300HP', 110000), 
                                 ('Western Star 57X', 'Class 8 Truck', 'Sleeper Cab, Manual Trans, 505HP', 165000), 
                                 ('eCascadia', 'Class 8 Truck', 'Electric, Day Cab, 470HP', 285000)], 
                      'price_range': (80000, 200000)
                     }, 
            
            'Bus': {'models': [('Transit Bus 40ft', 'City Bus', 'Low Floor, Diesel, 280HP', 285000), 
                               ('Transit Bus 60ft', 'City Bus', 'Articulated, Hybrid, 330HP', 425000), 
                               ('School Bus C2', 'School Bus', 'Type C, Diesel, 240HP', 155000), 
                               ('Coach Bus J4500', 'Coach Bus', 'Luxury, Diesel, 450HP', 485000), 
                               ('Electric Transit 35ft', 'City Bus', 'Electric, Low Floor, 350HP', 385000)], 
                    'price_range': (150000, 400000)
                   }, 
            
            'Powertrain': {'models': [('X15 Engine', 'Heavy Duty Engine', '15L, 605HP, EPA 2021', 45000), 
                                      ('X12 Engine', 'Heavy Duty Engine', '12L, 500HP, EPA 2021', 38000), 
                                      ('DD15 Engine', 'Heavy Duty Engine', '14.8L, 505HP, EPA 2021', 42000), 
                                      ('DT12 Transmission', 'Automated Manual', '12-Speed, Direct Drive', 28000), 
                                      ('Allison 3000', 'Automatic Trans', '6-Speed, Heavy Duty', 32000), 
                                      ('Meritor FUELite Tandem', 'Rear Axle', '40K lb, 2.26 Ratio', 22000)], 
                           'price_range': (20000, 80000)
                          }
           }

In [4]:
# Customer database
CUSTOMERS = {'Fleet': {'names': ['ABC Logistics', 'TransAmerica Freight', 'Swift Transportation', 'Prime Inc', 
                                 'Schneider National', 'JB Hunt Transport', 'Knight Transportation', 
                                 'Werner Enterprises', 'Covenant Transport', 'US Xpress', 'Marten Transport', 
                                 'Heartland Express'], 
                       'regions': ['US-Midwest', 'US-Southeast', 'US-West', 'US-Northeast', 'Canada-Ontario', 
                                   'Canada-Quebec'], 
                       'tiers': ['Platinum', 'Platinum', 'Gold', 'Gold', 'Silver', 'Silver', 'Bronze'], 
                       'fleet_sizes': [500, 1200, 2000, 800, 300, 150, 75]
                      }, 
             
             'Government': {'names': ['City Transit Authority', 'Metro Transportation', 'State DOT', 'County Schools', 
                                      'Municipal Services', 'Federal GSA', 'Provincial Transit', 'Regional Transport'], 
                            'regions': ['US-Midwest', 'US-Southeast', 'US-West', 'Canada-Ontario', 'Brazil-Sao Paulo'], 
                            'tiers': ['Platinum', 'Gold', 'Gold', 'Silver'], 
                            'fleet_sizes': [200, 400, 150, 300, 100]
                           }, 
             
             'Dealer': {'names': ['Premium Truck Sales', 'Metro Commercial Vehicles', 'Interstate Truck Center', 
                                  'Pacific Coast Motors', 'Midwest Heavy Duty', 'Southern States Trucks'], 
                        'regions': ['US-Midwest', 'US-Southeast', 'US-West', 'US-Northeast'], 
                        'tiers': ['Gold', 'Silver', 'Silver', 'Bronze'], 
                        'fleet_sizes': [None]
                       }, 
             
             'Individual': {'names': ['Owner-Operator ' + str(i) for i in range(1, 50)], 
                            'regions': ['US-Midwest', 'US-Southeast', 'US-West', 'US-Northeast', 'Canada-Ontario'], 
                            'tiers': ['Silver', 'Bronze', 'Bronze'], 
                            'fleet_sizes': [1, 2, 3, 5] 
                           }
            }

In [5]:
# Plants
PLANTS = {'Cleveland-OH': {'capacity': 100, 'specialization': ['Truck'], 'efficiency': 0.95}, 
          'Denton-TX': {'capacity': 120, 'specialization': ['Truck', 'Powertrain'], 'efficiency': 0.92}, 
          'Portland-OR': {'capacity': 80, 'specialization': ['Truck'], 'efficiency': 0.90}, 
          'Curitiba-Brazil': {'capacity': 90, 'specialization': ['Bus', 'Truck'], 'efficiency': 0.88}, 
          'Mississauga-ON': {'capacity': 70, 'specialization': ['Bus'], 'efficiency': 0.93}, 
          'High Point-NC': {'capacity': 85, 'specialization': ['Bus'], 'efficiency': 0.91}
         }

In [6]:
# Delay reasons mapping
DELAY_REASONS = {'Parts/Material': ['Semiconductor shortage', 'Steel allocation', 'Tire shortage', 
                                    'Battery cells unavailable', 'Wiring harness delay', 'Glass shortage'], 
                 'Engineering': ['Customer spec changes', 'Design revision required', 'Compliance update needed', 
                                 'Custom feature development', 'Safety standard update'], 
                 'Production': ['Line maintenance', 'Labor shortage', 'Quality hold', 'Capacity constraint', 
                                'Tool breakdown', 'Previous order overrun'], 
                 'Quality': ['Failed inspection', 'Rework required', 'Supplier quality issue', 'Paint defect', 
                             'Assembly error'], 
                 'Customer': ['Payment pending', 'Delivery postponed', 'Specification unclear', 
                              'Contract negotiation', 'Fleet readiness'], 
                 'Regulatory': ['EPA certification', 'DOT approval', 'Import permit', 'Safety compliance']
                }

In [7]:
def calculate_unified_delay(row: Dict, current_date: datetime) -> tuple:
    """ Unified delay calculation function that handles all order statuses. Returns (delay_days, is_delayed)."""
    requested_delivery_date = row['requested_delivery_date']
    status                  = row.get('order_status', 'Pending')
    # Guard
    if status == 'Canceled' and row.get('cancel_reason') != 'Delay':
        return 0, False          # not a delivery delay
    
    actual_delivery_date    = row.get('actual_delivery_date')
    cancellation_date       = row.get('cancellation_date')

    # choose effective date
    if status == 'Completed' and pd.notna(actual_delivery_date):
        effective_date = actual_delivery_date
    elif status == 'Canceled' and pd.notna(cancellation_date):
        effective_date = cancellation_date
    else:
        effective_date = current_date

    raw_delay  = (effective_date - requested_delivery_date).days
    delay_days = max(0, raw_delay)           # <-- **clamp here**
    is_delayed = delay_days > 0
    return delay_days, is_delayed

In [8]:
class OrderGenerator:
    def __init__(self):
        self.order_counter = 45821  # Starting number from example
        self.customer_db = self._initialize_customers()
        self.current_date = START_DATE
        
    def _initialize_customers(self) -> Dict:
        """Create customer database with consistent IDs"""
        customer_list = []
        cust_id = 10000
        
        for ctype, cdata in CUSTOMERS.items():
            for name in cdata['names']:
                customer_list.append({
                    'customer_id': f'CUST-{cust_id}' if ctype != 'Fleet' else f'FLEET-{cust_id}',
                    'customer_name': name,
                    'customer_type': ctype,
                    'customer_tier': np.random.choice(cdata['tiers']),
                    'customer_region': np.random.choice(cdata['regions']),
                    'fleet_size': np.random.choice([s for s in cdata['fleet_sizes'] if s is not None]) if cdata['fleet_sizes'][0] else None,
                    'previous_orders': 0,
                    'credit_rating': np.random.choice(['AAA', 'AA', 'A', 'BBB'], p=[0.2, 0.3, 0.35, 0.15])
                })
                cust_id += 1
                
        return pd.DataFrame(customer_list)
    
    def generate_order_batch(self, date: datetime, daily_orders: int) -> List[Dict]:
        """Generate a batch of orders for a specific date"""
        orders = []

        for _ in range(daily_orders):
            # Select customer with bias towards repeat customers
            if np.random.random() < 0.7 and len(self.customer_db[self.customer_db['previous_orders'] > 0]) > 0:
                customer = self.customer_db[self.customer_db['previous_orders'] > 0].sample(1).iloc[0]
            else:
                customer = self.customer_db.sample(1).iloc[0]

            # Determine number of line items (most orders have 1-2 items)
            num_items = np.random.choice([1, 2, 3, 4], p=[0.6, 0.25, 0.1, 0.05])

            # Generate order
            order_id = f'CV-{date.year}-{self.order_counter:06d}'
            self.order_counter += 1

            # Customer type influences product selection
            if customer['customer_type'] == 'Government':
                product_weights = {'Bus': 0.7, 'Truck': 0.2, 'Powertrain': 0.1}
            elif customer['customer_type'] == 'Fleet':
                product_weights = {'Truck': 0.75, 'Powertrain': 0.2, 'Bus': 0.05}
            else:
                product_weights = {'Truck': 0.6, 'Powertrain': 0.3, 'Bus': 0.1}

            order_value_total = 0
            base_production_days = 0

            for item_num in range(num_items):
                # Select product category
                product_cat = np.random.choice(list(product_weights.keys()), p=list(product_weights.values()))

                # Select specific model
                model_info = random.choice(PRODUCTS[product_cat]['models'])

                # Generate product ID
                if product_cat == 'Truck':
                    product_id = f'TR{70000 + np.random.randint(1, 9999)}'
                elif product_cat == 'Bus':
                    product_id = f'BS{20000 + np.random.randint(1, 9999)}'
                else:
                    product_id = f'PT{60000 + np.random.randint(1, 9999)}'

                # Quantity based on customer type and product
                if customer['customer_type'] == 'Individual':
                    quantity = 1
                elif customer['customer_type'] == 'Fleet' and product_cat == 'Truck':
                    quantity = np.random.choice([1, 5, 10, 25, 50], p=[0.2, 0.3, 0.25, 0.15, 0.1])
                elif customer['customer_type'] == 'Government' and product_cat == 'Bus':
                    quantity = np.random.choice([2, 5, 10, 20], p=[0.3, 0.35, 0.25, 0.1])
                else:
                    quantity = np.random.choice([1, 2, 3, 5], p=[0.5, 0.3, 0.15, 0.05])

                # Determine if new or pre-owned
                if np.random.random() < 0.85:  # 85% new vehicles
                    condition = 'New'
                    age = 0.0
                    miles_driven = np.random.randint(0, 50)  # Delivery miles only
                    depreciation = 0
                else:
                    condition = 'Pre-owned'
                    age = round(np.random.uniform(0.5, 5.0), 1)
                    miles_driven = int(age * np.random.uniform(15000, 45000))
                    depreciation = age * 0.15  # 15% per year depreciation

                # Pricing
                base_price = model_info[3]
                current_price = base_price * (1 - depreciation) if condition == 'Pre-owned' else base_price

                # Options and customization
                if product_cat != 'Powertrain' and customer['customer_tier'] in ['Platinum', 'Gold']:
                    options_value = base_price * np.random.uniform(0.02, 0.15)
                else:
                    options_value = base_price * np.random.uniform(0, 0.05)

                total_item_value = (current_price + options_value) * quantity
                order_value_total += total_item_value

                # Determine order priority and timeline
                is_rush = np.random.random() < 0.1
                rush_premium = total_item_value * 0.05 if is_rush else 0

                # Base production time
                if product_cat == 'Truck':
                    base_days = np.random.randint(90, 150)
                elif product_cat == 'Bus':
                    base_days = np.random.randint(120, 180)
                else:
                    base_days = np.random.randint(30, 60)

                if condition == 'Pre-owned':
                    base_days = int(base_days * 0.3)  # Much faster for pre-owned

                base_production_days = max(base_production_days, base_days)

                # Build complexity
                if options_value > base_price * 0.1:
                    build_complexity = 'Custom'
                elif options_value > base_price * 0.05:
                    build_complexity = 'Complex'
                else:
                    build_complexity = 'Standard'

                # Calculate a realistic customer requested date before creating the order dict
                if is_rush: 
                    # Rush orders: customer wants it ASAP, maybe unrealistic
                    customer_expectation_days = int(base_production_days * 0.7)  # 30% faster than standard
                else:
                    # Normal orders: customer gives reasonable buffer but not too much
                    customer_expectation_days = int(base_production_days * np.random.uniform(1.1, 1.3))  # 10-30% buffer over production time
                requested_delivery_date = date + timedelta(days=customer_expectation_days)

                # Create order record
                order = {'order_id': order_id, 
                         'order_date': date, 
                         'customer_id': customer['customer_id'], 
                         'product_id': product_id, 
                         'quantity': quantity,

                         # Product details 
                         'product_category': product_cat, 
                         'vehicle_model': model_info[0], 
                         'vehicle_class': model_info[1], 
                         'configuration': model_info[2], 
                         'color': np.random.choice(['White', 'Blue', 'Red', 'Black', 'Silver', 'Green', 'Yellow']), 
                         'condition': condition, 
                         'age': age, 
                         'miles_driven': miles_driven, 
                         'base_price': base_price, 
                         'current_price': current_price, 

                         # Financial 
                         'order_value_usd': total_item_value, 
                         'base_price_usd': base_price, 
                         'options_value_usd': options_value, 
                         'material_cost_usd': base_price * np.random.uniform(0.55, 0.65), 
                         'labor_cost_usd': base_price * np.random.uniform(0.15, 0.25), 
                         'shipping_cost_usd': np.random.uniform(500, 5000) if product_cat != 'Powertrain' else np.random.uniform(100, 500), 
                         'overhead_cost_usd': base_price * np.random.uniform(0.08, 0.12), 
                         'rush_order_premium_usd': rush_premium, 

                         # Customer details 
                         'customer_name': customer['customer_name'], 
                         'customer_type': customer['customer_type'], 
                         'customer_tier': customer['customer_tier'], 
                         'customer_region': customer['customer_region'], 
                         'fleet_size': customer['fleet_size'], 
                         'previous_orders_count': customer['previous_orders'], 
                         'customer_satisfaction': round(np.random.uniform(3.5, 5.0), 1) if customer['customer_tier'] == 'Platinum' else round(np.random.uniform(3.0, 4.8), 1), 
                         'sales_channel': np.random.choice(['Direct', 'Online', 'Dealership', 'Phone'], p=[0.4, 0.3, 0.2, 0.1]), 
                         'customer_credit_rating': customer['credit_rating'], 

                         # Build details 
                         'build_complexity': build_complexity, 
                         'base_production_days': base_production_days, 
                         'requested_delivery_date': requested_delivery_date,
                         'order_priority': 'Critical' if is_rush else np.random.choice(['High', 'Medium', 'Low'], p=[0.2, 0.5, 0.3])
                        }
                
                # ---- default lifecycle placeholders so downstream code never KeyErrors ----
                order.update({
                    'order_status': 'Pending',          # initial status
                    'actual_delivery_date': None,
                    'cancellation_date': None,
                    'actual_production_start_date': None,
                    'actual_production_end_date': None
                })


                orders.append(order)

            # Update customer order count
            self.customer_db.loc[self.customer_db['customer_id'] == customer['customer_id'], 'previous_orders'] += 1

        return orders

    def assign_production_details(self, order: Dict) -> Dict:
        """Assign production plant and schedule"""
        # Select appropriate plant
        eligible_plants = [p for p, d in PLANTS.items() if order['product_category'] in d['specialization']]
        
        # Consider customer region for plant selection
        if 'Brazil' in order['customer_region']:
            if 'Curitiba-Brazil' in eligible_plants:
                plant = 'Curitiba-Brazil'
            else:
                plant = np.random.choice(eligible_plants)
        elif 'Canada' in order['customer_region']:
            if 'Mississauga-ON' in eligible_plants:
                plant = 'Mississauga-ON'
            else:
                plant = np.random.choice(eligible_plants)
        else:
            plant = np.random.choice(eligible_plants)
        
        order['production_plant'] = plant
        
        # Production line assignment
        if order['product_category'] == 'Truck':
            order['production_line'] = np.random.choice(['Line-A', 'Line-B', 'Heavy-Line-1'])
        elif order['product_category'] == 'Bus':
            order['production_line'] = np.random.choice(['Bus-Line-1', 'Bus-Line-2'])
        else:
            order['production_line'] = np.random.choice(['Engine-Bay-1', 'Engine-Bay-2', 'Engine-Bay-3'])
        
        # Capacity calculations
        plant_capacity = PLANTS[plant]['capacity']
        current_utilization = np.random.uniform(0.7, 0.95)
        order['production_capacity_used'] = round((order['quantity'] / plant_capacity) * 100, 2)
        order['capacity_available_percent'] = round((1 - current_utilization) * 100, 2)
        
        # Production timing
        lead_time_days = order['base_production_days']
        order['planned_production_start_date'] = order['order_date'] + timedelta(days=np.random.randint(5, 15))
        order['planned_production_end_date'] = order['planned_production_start_date'] + timedelta(days=int(lead_time_days * 0.7))
        
        # Ensure planned/promised delivery date is usually after the requested delivery date:
        # More realistic, distributed promise date calculation
#         if order['customer_tier']=='Platinum' and np.random.random()<0.2:     # 20% chance of early delivery for Platinum customers
#             buffer_days = np.random.randint(-7, -1)                           # 1-7 days early
#         elif np.random.random() < 0.15:                                       # 15% chance to promise exactly on requested date
#             buffer_days = 0                                                   # On-time delivery
#         else:
#             buffer_days = np.random.randint(3, 45)                            # Most common: promise with 3-45 days safety buffer
#         order['planned_delivery_date'] = max(order['requested_delivery_date'] + timedelta(days=buffer_days), 
#                                              order['planned_production_end_date'] + timedelta(days=np.random.randint(3, 10)))
        # Calculate planned delivery based on production timeline + shipping
        min_delivery_date = order['planned_production_end_date'] + timedelta(days=np.random.randint(2, 7))  # Shipping time
        # Company promises based on realistic timelines, not customer wishes
        if order['customer_tier'] == 'Platinum' and np.random.random() < 0.3:
            # Platinum gets aggressive promises, sometimes beat customer request
            promise_buffer = np.random.randint(-7, 7)    # Can be slightly early or late
        elif order['customer_tier'] == 'Gold':
            promise_buffer = np.random.randint(0, 10)    # Small buffer
        else:
            promise_buffer = np.random.randint(0, 25)    # Larger buffer for lower tiers
        order['planned_delivery_date'] = max(min_delivery_date, min_delivery_date + timedelta(days=promise_buffer))
    
        # Engineering requirements
        if order['build_complexity'] == 'Custom' or (order['build_complexity'] == 'Complex' and np.random.random() < 0.5):
            order['custom_engineering_required'] = True
            order['engineering_status'] = np.random.choice(['Approved', 'Pending', 'In_Review'], p=[0.6, 0.25, 0.15])
            order['technical_complexity_score'] = np.random.randint(6, 10)
        else:
            order['custom_engineering_required'] = False
            order['engineering_status'] = 'Approved'
            order['technical_complexity_score'] = np.random.randint(1, 5)
        
        if order['engineering_status'] == 'Approved':
            order['engineering_approval_date'] = order['order_date'] + timedelta(days=np.random.randint(2, 10))
        else:
            order['engineering_approval_date'] = None
        
        order['design_changes_count'] = np.random.choice([0, 1, 2, 3], p=[0.6, 0.25, 0.1, 0.05])
        
        # Labor requirements
        if order['product_category'] == 'Truck':
            base_hours = 120
        elif order['product_category'] == 'Bus':
            base_hours = 200
        else:
            base_hours = 40
        
        order['labor_hours_required'] = int(base_hours * order['quantity'] * (1.2 if order['build_complexity'] == 'Custom' else 1.0))
        order['production_sequence'] = np.random.randint(1, 100)
        
        return order

    def assign_supply_chain_status(self, order: Dict) -> Dict:
        """Determine supply chain and inventory status"""
        # Higher tier customers and simpler builds have better parts availability
        if order['customer_tier'] == 'Platinum':
            parts_availability = np.random.uniform(0.92, 1.0)
        elif order['build_complexity'] == 'Standard':
            parts_availability = np.random.uniform(0.85, 1.0)
        else:
            parts_availability = np.random.uniform(0.70, 1.0)
        
        order['parts_availability'] = round(parts_availability * 100, 2)
        
        # Inventory shortages
        if parts_availability == 1.0:
            order['inventory_shortage_count'] = 0
            order['backorder_parts_count'] = 0
        else:
            order['inventory_shortage_count'] = np.random.choice([0, 1, 2, 3, 5, 8], p=[0.5, 0.2, 0.15, 0.1, 0.04, 0.01])
            order['backorder_parts_count'] = min(order['inventory_shortage_count'], np.random.randint(0, 3))
        
        # Lead times for components
        if order['product_category'] == 'Powertrain':
            order['longest_lead_time_days'] = np.random.choice([30, 45, 60, 90], p=[0.4, 0.3, 0.2, 0.1])
        else:
            order['longest_lead_time_days'] = np.random.choice([60, 90, 120, 180], p=[0.3, 0.3, 0.25, 0.15])
        
        # Sourcing
        order['component_sourcing_status'] = np.random.choice(['In_House', 'External', 'Mixed'], p=[0.3, 0.2, 0.5])
        
        # Supply risk
        if order['inventory_shortage_count'] > 0:
            order['supply_risk_score'] = round(np.random.uniform(5.0, 9.0), 1)
        else:
            order['supply_risk_score'] = round(np.random.uniform(1.0, 4.5), 1)
        
        # Key supplier issues
        if order['supply_risk_score'] > 7:
            order['key_supplier_issues'] = np.random.choice([
                'Steel shortage', 'Semiconductor allocation', 'Logistics delays',
                'Supplier bankruptcy', 'Quality issues', 'Labor strike'
            ])
        else:
            order['key_supplier_issues'] = None
        
        # Overall supply status
        if order['parts_availability'] >= 95 and order['inventory_shortage_count'] == 0:
            order['supply_chain_status'] = 'Ready'
        elif order['parts_availability'] >= 80 or order['inventory_shortage_count'] <= 2:
            order['supply_chain_status'] = 'Partial'
        else:
            order['supply_chain_status'] = 'Critical'
        
        return order
    
    def simulate_cancellations(self, order: Dict, current_date: datetime) -> Dict:
        """Simulate order cancellations based on realistic business scenarios"""
        # Base cancellation probability (2-3% of all orders)
        cancel_probability = 0.025

        # Increase probability based on risk factors
        if order['delay_impact_severity'] == 'Critical':
            cancel_probability += 0.15
        elif order['delay_impact_severity'] == 'High':
            cancel_probability += 0.08
        elif order['delay_impact_severity'] == 'Medium':
            cancel_probability += 0.03

        # Customer tier affects cancellation rate
        if order['customer_tier'] == 'Bronze':
            cancel_probability *= 1.5
        elif order['customer_tier'] == 'Silver':
            cancel_probability *= 1.2
        elif order['customer_tier'] == 'Platinum':
            cancel_probability *= 0.5

        # Large orders more likely to be canceled due to scrutiny
        if order['order_value_usd'] > 500000:
            cancel_probability += 0.02

        # Long delays increase cancellation risk
        if order['delay_days'] > 60:
            cancel_probability += 0.1
        elif order['delay_days'] > 30:
            cancel_probability += 0.05

        # Determine if order gets canceled
        is_canceled = np.random.random() < min(cancel_probability, 0.3)  # Cap at 30%

        if not is_canceled:
            order['cancel_reason'] = "N/A"
            order['cancellation_date'] = None
            return order

        # Determine cancellation reason
        cancel_reasons = ['Customer Reasons', 'Delay', 'Other']

        # Probability distribution based on delay severity
        if order['delay_impact_severity'] in ['Critical', 'High']:
            # More likely to be delay-related
            reason_probs = [0.25, 0.6, 0.15]
        elif order['delay_impact_severity'] in ['Medium', 'Low']:
            reason_probs = [0.4, 0.4, 0.2]
        else:
            # No delay severity (early cancellation)
            reason_probs = [0.5, 0.2, 0.3]

        cancel_reason = np.random.choice(cancel_reasons, p=reason_probs)

        # CONSTRAINT: If cancellation reason is 'Delay', ensure we can actually be late
        # Check if we can realistically have a delay-based cancellation
        requested_date = order['requested_delivery_date']
        earliest_possible_cancel = requested_date + timedelta(days=7)  # At least 1 week late

        if cancel_reason == 'Delay' and current_date < earliest_possible_cancel:
            # We can't have a realistic delay-based cancellation yet
            # Switch to customer reasons or other
            cancel_reason = np.random.choice(['Customer Reasons', 'Other'], p=[0.7, 0.3])

        order['cancel_reason'] = cancel_reason

        # Determine cancellation date based on the reason
        actual_start = order['actual_production_start_date']
        actual_end = order['actual_production_end_date']
        planned_start = order['planned_production_start_date']

        if cancel_reason == 'Delay':
            # DELAY-BASED CANCELLATION: Must be after requested delivery date
            min_cancel_date = max(
                requested_date + timedelta(days=np.random.randint(7, 30)),  # 1-4 weeks late
                order['order_date'] + timedelta(days=14)  # At least 2 weeks after order
            )

            # Maximum cancellation date based on how long customers typically wait
            if order['customer_tier'] == 'Platinum':
                max_wait_days = 45  # Premium customers don't wait as long
            elif order['customer_tier'] == 'Gold':
                max_wait_days = 60
            else:
                max_wait_days = 90  # Bronze/Silver customers more patient

            max_cancel_date = requested_date + timedelta(days=max_wait_days)

            # Also consider production stage
            if actual_start is not None:
                # If production started, cancellation is later
                if actual_end is not None:
                    # Production completed but still delayed
                    stage_min_date = actual_end + timedelta(days=np.random.randint(1, 14))
                else:
                    # During production
                    stage_min_date = actual_start + timedelta(days=np.random.randint(14, 45))

                min_cancel_date = max(min_cancel_date, stage_min_date)

        elif cancel_reason == 'Customer Reasons':
            # CUSTOMER-DRIVEN CANCELLATION: Can happen anytime, but usually early
            min_cancel_date = order['order_date'] + timedelta(days=np.random.randint(1, 21))  # 1-3 weeks
            max_cancel_date = requested_date + timedelta(days=np.random.randint(-7, 14))  # Might be before or slightly after request

            # Ensure minimum date is respected
            max_cancel_date = max(max_cancel_date, min_cancel_date)

        else:  # 'Other' internal reasons
            # INTERNAL ISSUE CANCELLATION: Can happen during production stages
            if actual_start is not None:
                if actual_end is not None:
                    # Issue discovered after production
                    min_cancel_date = actual_end + timedelta(days=np.random.randint(1, 7))
                    max_cancel_date = min_cancel_date + timedelta(days=np.random.randint(7, 21))
                else:
                    # Issue during production
                    min_cancel_date = actual_start + timedelta(days=np.random.randint(1, 30))
                    max_cancel_date = min_cancel_date + timedelta(days=np.random.randint(7, 30))
            else:
                # Issue before production (design, compliance, etc.)
                min_cancel_date = order['order_date'] + timedelta(days=np.random.randint(7, 30))
                max_cancel_date = planned_start if planned_start else min_cancel_date + timedelta(days=21)

        # Ensure cancellation date doesn't exceed current simulation date
        max_cancel_date = min(max_cancel_date, current_date)

        # Generate final cancellation date
        if min_cancel_date <= max_cancel_date:
            days_diff = max(1, (max_cancel_date - min_cancel_date).days)
            cancel_date = min_cancel_date + timedelta(days=np.random.randint(0, days_diff))
        else:
            cancel_date = min_cancel_date

        # FINAL VALIDATION: Ensure delay cancellations are actually late
        if cancel_reason == 'Delay' and cancel_date <= requested_date:
            # Force the cancellation to be late
            cancel_date = requested_date + timedelta(days=np.random.randint(7, 21))
            # But don't exceed current date
            cancel_date = min(cancel_date, current_date)

        order['cancellation_date'] = cancel_date

        # Update order status and delivery date
        order['order_status'] = 'Canceled'
        order['actual_delivery_date'] = None

        # Adjust delay-related fields based on cancellation reason
        if cancel_reason == 'Customer Reasons':
            # Customer canceled - clear delay info
            order['primary_delay_category'] = "N/A"
            order['delay_reason'] = "N/A"
            order['secondary_delay_category'] = "N/A"
            order['delay_impact_severity'] = "N/A"
            order['escalation_level'] = "N/A"
        elif cancel_reason == 'Other':
            # Internal non-delay issue - clear delay info
            order['primary_delay_category'] = "N/A"
            order['delay_reason'] = "N/A"
            order['secondary_delay_category'] = "N/A"
            order['delay_impact_severity'] = "N/A"
            order['escalation_level'] = "N/A"
        # For 'Delay' reason, keep existing delay information

        # Calculate revenue impact based on production stage
        base_order_value = order['order_value_usd']

        if actual_start is None or cancel_date < actual_start:
            # Canceled before production started
            costs_incurred = base_order_value * np.random.uniform(0.05, 0.10)
        elif actual_end is not None and cancel_date >= actual_end:
            # Canceled after production completed
            costs_incurred = base_order_value * np.random.uniform(0.70, 0.85)
        else:
            # Canceled during production
            costs_incurred = base_order_value * np.random.uniform(0.30, 0.70)

        # Total revenue impact = lost sale + wasted costs
        order['revenue_impact_usd'] = base_order_value + costs_incurred

        return order
    
    # Add realistic revenue impact for non-canceled delayed orders
    def add_realistic_revenue_impact(self, order: Dict) -> Dict:
        """Add realistic revenue impact for delayed orders"""

        if order['order_status'] == 'Canceled':
            return order  # Revenue impact already calculated in cancellation logic

        if not order['is_delayed'] or order['delay_days'] <= 0:
            order['revenue_impact_usd'] = 0
            return order

        # 5-8% of delayed orders have measurable revenue impact
        has_revenue_impact = np.random.random() < 0.07

        if not has_revenue_impact:
            order['revenue_impact_usd'] = 0
            return order

        # Calculate impact based on severity and customer tier
        base_order_value = order['order_value_usd']

        if order['delay_impact_severity'] == 'Critical':
            impact_rate = np.random.uniform(0.05, 0.15)  # 5-15% of order value
        elif order['delay_impact_severity'] == 'High':
            impact_rate = np.random.uniform(0.02, 0.08)   # 2-8% of order value
        elif order['delay_impact_severity'] == 'Medium':
            impact_rate = np.random.uniform(0.01, 0.04)   # 1-4% of order value
        else:
            impact_rate = np.random.uniform(0.005, 0.02)  # 0.5-2% of order value

        # Higher impact for premium customers (they have more leverage)
        if order['customer_tier'] == 'Platinum':
            impact_rate *= 1.5
        elif order['customer_tier'] == 'Gold':
            impact_rate *= 1.2

        # Types of revenue impact: late delivery penalties, expedited shipping, storage costs
        revenue_impact = base_order_value * impact_rate

        # Add storage costs for long delays
        if order['delay_days'] > 30:
            storage_cost = order['delay_days'] * np.random.uniform(100, 300)
            revenue_impact += storage_cost

        order['revenue_impact_usd'] = round(revenue_impact, 2)

        return order

    def simulate_order_execution(self, order: Dict, current_date: datetime) -> Dict:
        """Simulate order execution and delays"""
        # Calculate initial delay status for ALL orders
        #order['delay_days'], order['is_delayed'] = calculate_unified_delay(order, current_date)

        # Only process orders that should have started production
        if current_date < order['planned_production_start_date']:
            order['order_status'] = 'Pending'
            order['actual_production_start_date'] = None
            order['actual_production_end_date'] = None
            order['actual_delivery_date'] = None
            order['cancellation_date'] = None  # Initialize this too
            
            order['delay_days'], order['is_delayed'] = calculate_unified_delay(order, current_date)
            return order

        # Determine if order will have delays
        delay_probability = 0.15  # Base 15% delay rate (realistic for manufacturing)

        # Adjust probability based on factors
        if order['engineering_status'] != 'Approved':
            delay_probability += 0.3
        if order['supply_chain_status'] == 'Critical':
            delay_probability += 0.25
        elif order['supply_chain_status'] == 'Partial':
            delay_probability += 0.1
        if order['customer_tier'] == 'Platinum':
            delay_probability *= 0.35  # Less than half as likely to be delayed
        if order['build_complexity'] == 'Custom':
            delay_probability += 0.15

        has_delay = np.random.random() < min(delay_probability, 0.8)

        if has_delay:
            # Determine delay category and severity
            if order['engineering_status'] != 'Approved' and np.random.random() < 0.6:
                primary_category = 'Engineering'
                severity = np.random.choice(['Medium', 'High', 'Critical'], p=[0.3, 0.5, 0.2])
            elif order['supply_chain_status'] in ['Critical', 'Partial'] and np.random.random() < 0.7:
                primary_category = 'Parts/Material'
                severity = 'Critical' if order['supply_chain_status'] == 'Critical' else np.random.choice(['Medium', 'High'], p=[0.4, 0.6])
            else:
                primary_category = np.random.choice(list(DELAY_REASONS.keys()), p=[0.4, 0.25, 0.2, 0.1, 0.03, 0.02])
                severity = np.random.choice(['Low', 'Medium', 'High', 'Critical'], p=[0.2, 0.4, 0.3, 0.1])

            # Calculate delay days based on severity
            if severity == 'Critical':
                delay_days = np.random.randint(30, 90)
            elif severity == 'High':
                delay_days = np.random.randint(15, 30)
            elif severity == 'Medium':
                delay_days = np.random.randint(7, 15)
            else:
                delay_days = np.random.randint(1, 7)

            order['primary_delay_category'] = primary_category
            order['delay_reason'] = np.random.choice(DELAY_REASONS[primary_category])
            order['delay_impact_severity'] = severity

            # Secondary delays (20% chance)
            if np.random.random() < 0.2:
                remaining_categories = [c for c in DELAY_REASONS.keys() if c != primary_category]
                order['secondary_delay_category'] = np.random.choice(remaining_categories)
            else:
                order['secondary_delay_category'] = None

            # Production impacts
            order['actual_production_start_date'] = order['planned_production_start_date'] + timedelta(days=np.random.randint(0, max(1, math.ceil(delay_days/1.5))))
            production_time = (order['planned_production_end_date'] - order['planned_production_start_date']).days
            order['actual_production_end_date'] = order['actual_production_start_date'] + timedelta(days=int(production_time * np.random.uniform(1.0, 1.2)))

            # Calculate actual delivery date
            min_delivery_date = order['actual_production_end_date'] + timedelta(days=np.random.randint(1, 4))
            planned_delivery_with_delay = order['planned_delivery_date'] + timedelta(days=delay_days)
            order['actual_delivery_date'] = max(min_delivery_date, planned_delivery_with_delay)

        else:
            # No delays - might even be early
            order['primary_delay_category'] = None
            order['delay_reason'] = None
            order['secondary_delay_category'] = None
            order['delay_impact_severity'] = None

            # Small chance of early delivery
            if np.random.random() < 0.15 and order['customer_tier'] in ['Platinum', 'Gold']:
                early_days = np.random.randint(1, 7)
                order['actual_production_start_date'] = order['planned_production_start_date']
                order['actual_production_end_date'] = order['planned_production_end_date'] - timedelta(days=early_days)
                order['actual_delivery_date'] = order['actual_production_end_date'] + timedelta(days=np.random.randint(1, 3))
            else:
                order['actual_production_start_date'] = order['planned_production_start_date']
                order['actual_production_end_date'] = order['planned_production_end_date']
                order['actual_delivery_date'] = order['planned_delivery_date']

        # CHECK: If any actual dates are in the future, adjust status and clear future dates
        if order['actual_delivery_date'] and order['actual_delivery_date'] > current_date:
            # Delivery hasn't happened yet
            order['actual_delivery_date'] = None
            if order['actual_production_end_date'] and order['actual_production_end_date'] > current_date:
                # Production not finished
                order['actual_production_end_date'] = None
                if order['actual_production_start_date'] and order['actual_production_start_date'] > current_date:
                    # Production not started
                    order['actual_production_start_date'] = None
                    order['order_status'] = 'Pending'
                else:
                    # Production started but not finished
                    order['order_status'] = 'In_Production'
            else:
                # Production finished but not delivered
                order['order_status'] = 'In_Production'
        else:
            # Order completed
            order['order_status'] = 'Completed'

        # Use unified delay calculation for ALL orders
        order['delay_days'], order['is_delayed'] = calculate_unified_delay(order, current_date)

        # Quality and resolution
        if order['is_delayed']:
            order['constraint_resolved'] = order['order_status'] == 'Completed'
            if order['constraint_resolved']:
                order['resolution_days'] = max(5, abs(order['delay_days']) - 10)
            else:
                order['resolution_days'] = 0

            # Escalation based on severity (only if there was a delay category)
            if order['delay_impact_severity']:
                if order['delay_impact_severity'] == 'Critical':
                    order['escalation_level'] = 'Director'
                elif order['delay_impact_severity'] == 'High':
                    order['escalation_level'] = 'Manager'
                elif order['delay_impact_severity'] == 'Medium':
                    order['escalation_level'] = 'Supervisor'
                else:
                    order['escalation_level'] = 'None'
            else:
                order['escalation_level'] = None
        else:
            order['constraint_resolved'] = None
            order['resolution_days'] = None
            order['escalation_level'] = None

        # Quality checks
        if order['order_status'] in ['In_Production', 'Completed']:
            order['quality_check_status'] = np.random.choice(['Passed', 'Failed', 'In_Progress'], p=[0.85, 0.05, 0.1])
            if order['quality_check_status'] == 'Passed':
                order['quality_score'] = round(np.random.uniform(92, 99), 1)
                order['rework_required'] = False
            elif order['quality_check_status'] == 'Failed':
                order['quality_score'] = round(np.random.uniform(75, 85), 1)
                order['rework_required'] = True
            else:
                order['quality_score'] = None
                order['rework_required'] = None
        else:
            order['quality_check_status'] = 'Pending'
            order['quality_score'] = None
            order['rework_required'] = None

        # Simulate potential cancellations
        order = self.simulate_cancellations(order, current_date)

        # If order was canceled, skip remaining processing
#         if order['order_status'] == 'Canceled':
#             return order
        # Refresh delay fields now that status/dates may have changed
        order['delay_days'], order['is_delayed'] = calculate_unified_delay(order, current_date)
        
        # --- ensure every late order has a primary reason -------------------------
        if order['is_delayed'] and order.get('primary_delay_category') in [None, 'N/A']:
            order['primary_delay_category'] = 'Production'  # or 'Schedule'
            order['delay_reason']           = 'Behind schedule'
            # choose a severity based on lateness
            if order['delay_days'] > 60:
                sev = 'Critical'
            elif order['delay_days'] > 30:
                sev = 'High'
            elif order['delay_days'] > 14:
                sev = 'Medium'
            else:
                sev = 'Low'
            order['delay_impact_severity']  = sev
            order['secondary_delay_category'] = 'N/A'

        return order

    def add_financial_details(self, order: Dict) -> Dict:
        """Add financial calculations and terms"""
        # Discounts based on customer tier and volume
        base_discount = 0
        if order['customer_tier'] == 'Platinum':
            base_discount = np.random.uniform(5, 8)
        elif order['customer_tier'] == 'Gold':
            base_discount = np.random.uniform(3, 5)
        elif order['customer_tier'] == 'Silver':
            base_discount = np.random.uniform(1, 3)

        # Volume discounts
        if order['quantity'] >= 50:
            base_discount += 2
        elif order['quantity'] >= 25:
            base_discount += 1
        elif order['quantity'] >= 10:
            base_discount += 0.5

        order['discount'] = round(base_discount, 2)

        # Rebates
        if order['customer_type'] == 'Government':
            order['rebate_usd'] = order['order_value_usd'] * np.random.uniform(0.02, 0.04)
        elif order['quantity'] >= 25:
            order['rebate_usd'] = order['order_value_usd'] * np.random.uniform(0.01, 0.03)
        else:
            order['rebate_usd'] = 0

        # Calculate profit margin
        total_cost = (order['material_cost_usd'] + order['labor_cost_usd'] + 
                     order['shipping_cost_usd'] + order['overhead_cost_usd'])
        revenue_after_discount = order['order_value_usd'] * (1 - order['discount']/100) - order['rebate_usd']
        order['profit_margin_percent'] = round(((revenue_after_discount - total_cost) / revenue_after_discount) * 100, 2)

        # Payment terms
        if order['customer_credit_rating'] == 'AAA':
            order['payment_terms'] = 'Net 60'
            order['payment_terms_days'] = 60
        elif order['customer_credit_rating'] == 'AA':
            order['payment_terms'] = 'Net 30'
            order['payment_terms_days'] = 30
        elif order['customer_credit_rating'] == 'A':
            order['payment_terms'] = '50% Advance'
            order['payment_terms_days'] = 0
        else:
            order['payment_terms'] = 'Payment on Delivery'
            order['payment_terms_days'] = 0

        # Currency
        if 'Brazil' in order['customer_region']:
            order['currency'] = 'BRL'
        elif 'Canada' in order['customer_region']:
            order['currency'] = 'CAD'
        else:
            order['currency'] = 'USD'

        # Additional costs for delays
        if order['is_delayed'] and order['delay_days'] > 0:
            # Storage costs
            order['storage_holding_cost_usd'] = order['delay_days'] * np.random.uniform(50, 200)

            # Late delivery penalties
            if order['delay_days'] > 30 and order['customer_tier'] in ['Platinum', 'Gold']:
                order['late_delivery_penalty_usd'] = order['order_value_usd'] * 0.02 * (order['delay_days'] / 30)
            else:
                order['late_delivery_penalty_usd'] = 0
        else:
            order['storage_holding_cost_usd'] = 0
            order['late_delivery_penalty_usd'] = 0

        # Keep any impact already calculated (e.g., cancellations)
        if 'revenue_impact_usd' not in order:
            order['revenue_impact_usd'] = 0

        return order

    def add_temporal_features(self, order: Dict) -> Dict:
        """Add time-based features"""
        order_date = order['order_date']
        
        order['order_year'] = order_date.year
        order['order_month'] = order_date.month
        order['order_quarter'] = f'Q{(order_date.month-1)//3 + 1}-{order_date.year}'
        
        # Season
        month = order_date.month
        if month in [3, 4, 5]:
            order['season'] = 'Spring'
        elif month in [6, 7, 8]:
            order['season'] = 'Summer'
        elif month in [9, 10, 11]:
            order['season'] = 'Fall'
        else:
            order['season'] = 'Winter'
        
        # Holiday periods
        holiday_months = [11, 12, 1, 7]  # November, December, January, July
        order['is_holiday_period'] = month in holiday_months
        
        # Demand factors (seasonal variations)
        demand_factors = {'Spring': 1.2, 'Summer': 1.0, 'Fall': 1.1, 'Winter': 0.8}
        if order['product_category'] == 'Bus' and month in [6, 7, 8]:  # School bus orders peak in summer
            order['demand_factor'] = 1.3
        else:
            order['demand_factor'] = demand_factors[order['season']]
        
        # Cycle time calculation
        if order['actual_delivery_date']:
            order['cycle_time_days'] = (order['actual_delivery_date'] - order['order_date']).days
        else:
            order['cycle_time_days'] = None
        
        # On-time delivery
        if order['actual_delivery_date'] and order['planned_delivery_date']:
            order['on_promise_delivery'] = order['actual_delivery_date'] <= order['planned_delivery_date']
        else:
            order['on_promise_delivery'] = None
        
        return order

    def add_promotional_features(self, order: Dict) -> Dict:
        """Add promotional information"""
        # Determine if promotion was available
        promo_probability = 0.3  # 30% of orders have promotions available
        
        # Seasonal promotions
        if order['season'] == 'Winter':
            promo_probability += 0.1
        if order['order_month'] in [11, 12]:  # Year-end
            promo_probability += 0.15
        
        order['promo_available'] = np.random.random() < promo_probability
        
        if order['promo_available']:
            # Promotion categories
            if order['order_month'] in [10, 11, 12]:
                promo_categories = ['Festival', 'Clearance', 'Year-End', 'Trade-in']
                promo_probs = [0.3, 0.3, 0.3, 0.1]
            elif order['order_month'] in [5, 7, 9]:  # Memorial Day, July 4th, Labor Day
                promo_categories = ['Federal Holiday', 'Summer Special', 'Trade-in']
                promo_probs = [0.5, 0.3, 0.2]
            elif order['customer_type'] == 'Government':
                promo_categories = ['Government', 'Volume', 'Trade-in']
                promo_probs = [0.6, 0.3, 0.1]
            else:
                promo_categories = ['Trade-in', 'Volume', 'Clearance']
                promo_probs = [0.4, 0.3, 0.3]
            
            order['promo_category'] = np.random.choice(promo_categories, p=promo_probs)
            
            # Promotion values
            if order['promo_category'] == 'Clearance':
                order['promo_value_discount'] = round(np.random.uniform(3, 6), 2)
                order['promo_value_rebate_usd'] = 0
            elif order['promo_category'] == 'Trade-in':
                order['promo_value_discount'] = 0
                order['promo_value_rebate_usd'] = np.random.choice([2500, 5000, 7500, 10000])
            else:
                order['promo_value_discount'] = round(np.random.uniform(1.5, 4), 2)
                order['promo_value_rebate_usd'] = np.random.choice([0, 1000, 2000, 3000], p=[0.4, 0.3, 0.2, 0.1])
            
            # Whether promotion was applied
            apply_prob = 0.5
            if order['customer_tier'] in ['Platinum', 'Gold']:
                apply_prob += 0.2
            if order['promo_value_discount'] > 3 or order['promo_value_rebate_usd'] > 5000:
                apply_prob += 0.2
            if order['sales_channel'] in ['Direct', 'Dealership']:
                apply_prob += 0.1

            order['promo_applied'] = np.random.random() < min(apply_prob, 0.9)
                
        else:
            order['promo_category'] = None
            order['promo_value_discount'] = 0
            order['promo_value_rebate_usd'] = 0
            order['promo_applied'] = 'N/A'
        
        return order

    def add_delivery_details(self, order: Dict) -> Dict:
        """Add delivery-specific information"""
        # Delivery region might differ from customer region
        if np.random.random() < 0.8:  # 80% delivered to customer region
            order['delivery_region'] = order['customer_region']
        else:
            # Cross-region delivery
            regions = ['US-Midwest', 'US-Southeast', 'US-West', 'US-Northeast', 
                      'Canada-Ontario', 'Canada-Quebec', 'Brazil-Sao Paulo']
            order['delivery_region'] = np.random.choice([r for r in regions if r != order['customer_region']])
        
        # Calculate delivery distance
        distance_matrix = {
            ('US-Midwest', 'US-Midwest'): (50, 500),
            ('US-Midwest', 'US-Southeast'): (600, 1200),
            ('US-Midwest', 'US-West'): (1500, 2200),
            ('US-Midwest', 'US-Northeast'): (700, 1300),
            ('US-Midwest', 'Canada-Ontario'): (300, 800),
            ('Canada-Ontario', 'Canada-Quebec'): (300, 600),
            ('Brazil-Sao Paulo', 'Brazil-Sao Paulo'): (50, 300),
        }
        
        # Find applicable distance range
        key = (order['production_plant'].split('-')[-1], order['delivery_region'].split('-')[-1])
        if key in distance_matrix:
            min_dist, max_dist = distance_matrix[key]
        else:
            min_dist, max_dist = 1000, 3000  # Default for cross-country
        
        order['delivery_distance'] = round(np.random.uniform(min_dist, max_dist), 1)
        
        # Delivery method based on distance and product
        if order['delivery_distance'] < 200:
            order['delivery_method'] = np.random.choice(['Customer_Pickup', 'Truck'], p=[0.3, 0.7])
        elif order['delivery_distance'] < 1000:
            order['delivery_method'] = 'Truck'
        elif order['delivery_distance'] < 2000:
            order['delivery_method'] = np.random.choice(['Truck', 'Rail'], p=[0.6, 0.4])
        else:
            order['delivery_method'] = np.random.choice(['Rail', 'Ship'], p=[0.7, 0.3])
        
        return order

In [9]:
def cleanup_data_values(df):
    """Clean up all None/NaN values and ensure proper data types"""
    
    print("\nCleaning up data values...")
    
    # Handle date columns first - convert None to NaT
    date_columns = ['actual_delivery_date', 'actual_production_start_date', 'actual_production_end_date', 
                    'engineering_approval_date', 'constraint_start_date', 'expected_resolution_date', 
                    'cancellation_date']
    
    for col in date_columns:
        if col in df.columns:
            # Convert to datetime, None becomes NaT automatically
            df[col] = pd.to_datetime(df[col])    # , errors='coerce'
    
    # Define categorical columns that should be "N/A" for specific conditions
    delay_categorical_columns = ['delay_reason', 'primary_delay_category', 'secondary_delay_category', 
                                 'delay_impact_severity', 'escalation_level']
    
    # Define numerical columns that should remain NaN when not applicable
    delay_numerical_columns = ['resolution_days']
    
    # Define other numerical columns that should remain NaN
    incomplete_numerical_columns = ['cycle_time_days', 'quality_score']
    
    # Define categorical columns that should be "N/A" for incomplete orders
    incomplete_categorical_columns = ['quality_check_status']
    
    # Define general categorical columns
    general_categorical_columns = ['key_supplier_issues', 'promo_category']
    
    # Define boolean-like columns that become "N/A" when not applicable
    boolean_like_columns = ['constraint_resolved', 'rework_required', 'on_promise_delivery']
    
    # 1. Clean up delay-related CATEGORICAL columns
    for col in delay_categorical_columns:
        if col in df.columns:
            # Set to "N/A" for non-delayed orders
            df.loc[df['is_delayed'] == False, col] = "N/A"
            # Set to "N/A" for canceled orders with customer/other reasons
            df.loc[(df['order_status'] == 'Canceled') & 
                   (df['cancel_reason'].isin(['Customer Reasons', 'Other'])), col] = "N/A"
            # Clean up None values
            df[col] = df[col].fillna("N/A")
            df[col] = df[col].replace([None, 'None'], "N/A")
    
    # 2. Handle delay-related NUMERICAL columns (keep as NaN)
    for col in delay_numerical_columns:
        if col in df.columns:
            # Set to NaN for non-delayed orders
            df.loc[df['is_delayed'] == False, col] = np.nan
            # Set to NaN for canceled orders with customer/other reasons
            df.loc[(df['order_status'] == 'Canceled') & 
                   (df['cancel_reason'].isin(['Customer Reasons', 'Other'])), col] = np.nan
            # Ensure it's float type to handle NaN properly
            df[col] = pd.to_numeric(df[col])    # , errors='coerce'
    
    # 3. Handle incomplete order NUMERICAL columns (keep as NaN)
    for col in incomplete_numerical_columns:
        if col in df.columns:
            # Keep as NaN for incomplete orders - don't force to "N/A"
            df[col] = pd.to_numeric(df[col])    # , errors='coerce'
    
    # 4. Handle incomplete order CATEGORICAL columns
    for col in incomplete_categorical_columns:
        if col in df.columns:
            df.loc[df['order_status'] == 'Canceled', col] = "N/A"
            df[col] = df[col].fillna("N/A")
            df[col] = df[col].replace([None, 'None'], "N/A")
    
    # 5. Clean up general categorical columns
    for col in general_categorical_columns:
        if col in df.columns:
            df[col] = df[col].fillna("N/A")
            df[col] = df[col].replace([None, 'None'], "N/A")
    
    # 6. Handle boolean-like columns that should become "N/A" when not applicable
    for col in boolean_like_columns:
        if col in df.columns:
            if col == 'constraint_resolved':
                # Set to "N/A" for non-delayed orders
                df.loc[df['is_delayed'] == False, col] = "N/A"
            elif col == 'rework_required':
                # Set to "N/A" for pending/canceled quality checks
                df.loc[df['quality_check_status'].isin(['Pending', 'In_Progress', 'N/A']), col] = "N/A"
            elif col == 'on_promise_delivery':
                # Set to "N/A" for incomplete orders
                df.loc[df['order_status'].isin(['Pending', 'Canceled']), col] = "N/A"
            
            # Clean up remaining None values
            df[col] = df[col].fillna("N/A")
            df[col] = df[col].replace([None, 'None'], "N/A")
            # Convert to object type to handle mixed True/False/"N/A"
            df[col] = df[col].astype('object')
    
    # 7. Special handling for quality-related fields
    if 'quality_check_status' in df.columns:
        # Set quality_score to NaN (not "N/A") when quality check is pending/in progress
        df.loc[df['quality_check_status'].isin(['Pending', 'In_Progress']), 'quality_score'] = np.nan
        # Ensure quality_score is numeric
        df['quality_score'] = pd.to_numeric(df['quality_score'])    # , errors='coerce'
    
    # 8. Ensure delay_days is 0 (not NaN) for non-delayed orders
    if 'delay_days' in df.columns:
        df.loc[df['is_delayed'] == False, 'delay_days'] = 0
        df['delay_days'] = pd.to_numeric(df['delay_days'])    # , errors='coerce'
    
    # 9. Ensure revenue_impact_usd is 0 (not NaN) for non-delayed/non-canceled orders
    if 'revenue_impact_usd' in df.columns:
        df.loc[(df['is_delayed'] == False) & (df['order_status'] != 'Canceled'), 'revenue_impact_usd'] = 0
        df['revenue_impact_usd'] = pd.to_numeric(df['revenue_impact_usd'])    # , errors='coerce'
    
    # 10. Handle other potential numerical columns that should remain NaN when not applicable
    other_numerical_columns = [
        'storage_holding_cost_usd', 'late_delivery_penalty_usd', 
        'production_capacity_used', 'capacity_available_percent',
        'labor_hours_required', 'technical_complexity_score',
        'design_changes_count', 'inventory_shortage_count',
        'backorder_parts_count', 'longest_lead_time_days',
        'supply_risk_score', 'previous_constraints_count',
        'days_until_requirement', 'customer_revenue_ytd',
        'competing_orders_count', 'parts_availability'
    ]
    
    for col in other_numerical_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col])    # , errors='coerce'
    
    print("Data cleanup completed!")
    print("- Categorical columns: None/NaN → 'N/A'")
    print("- Numerical columns: Kept as NaN where appropriate")
    print("- Date columns: None → NaT")
    
    return df

In [10]:
def generate_dataset(START_DATE, END_DATE, TARGET_ROWS):
    """Generate the complete dataset"""
    print("Starting dataset generation...")
    generator = OrderGenerator()
    all_orders = []
    
    # Generate orders day by day
    current_date = START_DATE
    total_days = (END_DATE - START_DATE).days
    orders_per_day = TARGET_ROWS / total_days
    
    order_count = 0
    # For realistic patterns: Not every day has orders
    # Probability of having orders on any given day
    if total_days <= 30:  # For testing with short date ranges
        order_probability = 0.5  # Base probability (50% on weekdays, 15% on weekends)
    else:  # For production with full date range
        order_probability = 0.7  # Base probability (70% on weekdays, 21% on weekends)

    while current_date <= END_DATE:
        # Determine if we have orders today
        is_weekend = current_date.weekday() in [5, 6]
        has_orders_today = np.random.random() < (order_probability * 0.3 if is_weekend else order_probability)

        if has_orders_today:
            # Calculate orders for today
            if is_weekend:
                daily_orders = max(1, int(np.random.poisson(orders_per_day * 0.5)))
            else:
                daily_orders = max(1, int(np.random.poisson(orders_per_day * 1.2)))
        
            # Generate orders for this day
            day_orders = generator.generate_order_batch(current_date, daily_orders)

            # Process each order through the pipeline
            for order in day_orders:
                order = generator.assign_production_details(order)
                order = generator.assign_supply_chain_status(order)
                order = generator.simulate_order_execution(order, END_DATE)  # Simulate as if viewing on end date
                order = generator.add_financial_details(order)
                order = generator.add_realistic_revenue_impact(order)
                order = generator.add_temporal_features(order)
                order = generator.add_promotional_features(order)
                order = generator.add_delivery_details(order)

                all_orders.append(order)
                order_count += 1
        
        current_date += timedelta(days=1)
        
        # Progress indicator
        if current_date.day == 1:
            print(f"Processing {(current_date - timedelta(days=1)).strftime('%B %Y')}...")
    
    print(f"\nGenerated {order_count} order line items")
    
    # Create DataFrame
    df = pd.DataFrame(all_orders)
    # Clean up data values
    df = cleanup_data_values(df)
            
    # Debug: Check what columns we actually have
    print(f"\nDataFrame shape: {df.shape}")
    if 'customer_id' not in df.columns:
        print("ERROR: 'customer_id' column not found!")
        print(f"Available columns: {sorted(df.columns.tolist())}")
        return df
    
    # Convert date columns to datetime if they aren't already
    date_columns = ['order_date', 'requested_delivery_date', 'planned_delivery_date', 'actual_delivery_date', 
                    'planned_production_start_date', 'actual_production_start_date', 'planned_production_end_date', 
                    'actual_production_end_date', 'engineering_approval_date', 'constraint_start_date', 
                    'expected_resolution_date', 'cancellation_date']
    
    for col in date_columns:
        if col in df.columns and 'datetime' not in str(df[col].dtype):
            df[col] = pd.to_datetime(df[col])    # , errors='coerce'
    
    # Add constraint tracking columns as suggested
    df['previous_constraints_count'] = df.groupby('customer_id')['is_delayed'].cumsum()
    df['constraint_start_date'] = df.apply(lambda x: x['planned_production_start_date'] if x['is_delayed'] else None, axis=1)
    df['expected_resolution_date'] = df.apply(lambda x: x['actual_delivery_date'] if x['is_delayed'] else None, axis=1)
    
    # Calculate days until requirement
    df['days_until_requirement'] = df.apply(lambda x: (x['requested_delivery_date'] - END_DATE).days if pd.notnull(x['requested_delivery_date']) else None, axis=1)
    
    # Customer revenue YTD
    df['customer_revenue_ytd'] = df.groupby(['customer_id', 'order_year'])['order_value_usd'].cumsum()
    
    # Competing orders count (orders in same plant/line in same month)
    df['competing_orders_count'] = df.groupby(['production_plant', 'production_line', 'order_year', 'order_month']).cumcount()
    
    # Data quality checks
    print("\nRunning data quality checks...")
    
    # Check 1: Product consistency
    product_consistency = df.groupby('product_id').agg({'product_category': 'nunique', 'base_price': 'nunique'})
    assert all(product_consistency['product_category'] == 1), "Product category inconsistency detected"
    
    # Check 2: Delivery date logic
    delivery_check = df[df['actual_delivery_date'].notna()]
    invalid_deliveries = delivery_check[delivery_check['actual_delivery_date'] < delivery_check['actual_production_end_date']]
    if len(invalid_deliveries) > 0:
        print(f"\nFound {len(invalid_deliveries)} orders with delivery before production end:")
        print(invalid_deliveries[['order_id', 'actual_production_end_date', 'actual_delivery_date', 'delay_days', 'is_delayed']].head())
        assert False, "Delivery before production end detected"
    
    # Check 3: Delay calculation (customer-centric)
    delay_calc_check = df[df['requested_delivery_date'].notna()].copy()

    # Calculate expected delay using the same logic as calculate_unified_delays
    def calculate_expected_delay(row, end_date):
        if row['order_status'] == 'Completed' and pd.notna(row['actual_delivery_date']):
            effective_date = row['actual_delivery_date']
        elif row['order_status'] == 'Canceled' and pd.notna(row['cancellation_date']):
            effective_date = row['cancellation_date']
        else:
            # In-progress or pending orders - use simulation end date
            effective_date = end_date
        return max(0, (effective_date - row['requested_delivery_date']).days)

    delay_calc_check['calculated_delay'] = delay_calc_check.apply(lambda row: calculate_expected_delay(row, END_DATE), axis=1)
    # Skip non‑delay cancellations when recomputing expected delay
    delay_calc_check = delay_calc_check[~((delay_calc_check['order_status'] == 'Canceled') & 
                                          (delay_calc_check['cancel_reason'] != 'Delay'))].copy()

    # Find mismatches for debugging
    mismatches = delay_calc_check[delay_calc_check['delay_days'] != delay_calc_check['calculated_delay']]

    if len(mismatches) > 0:
        print(f"\nFound {len(mismatches)} delay calculation mismatches:")

        # Debug columns
        debug_cols = ['order_id', 'order_status', 'cancel_reason', 'requested_delivery_date', 
                      'actual_delivery_date', 'cancellation_date', 'delay_days', 'calculated_delay', 'is_delayed']

        print("Sample mismatches:")
        print(mismatches[debug_cols].head(10))

        # Analyze by order status
        print(f"\nMismatches by order status:")
        print(mismatches['order_status'].value_counts())

        if 'Canceled' in mismatches['order_status'].values:
            canceled_mismatches = mismatches[mismatches['order_status'] == 'Canceled']
            print(f"\nCanceled order mismatches by cancel reason:")
            print(canceled_mismatches['cancel_reason'].value_counts())

            # Check for delay cancellations that might have timing issues
            delay_cancels = canceled_mismatches[canceled_mismatches['cancel_reason'] == 'Delay']
            if len(delay_cancels) > 0:
                print(f"\nDelay cancellations - checking timing logic:")
                for _, row in delay_cancels.head(5).iterrows():
                    req_date = row['requested_delivery_date']
                    cancel_date = row['cancellation_date']
                    delay_days = row['delay_days']
                    calc_delay = row['calculated_delay']
                    print(f"Order {row['order_id']}: Req={req_date.date()}, Cancel={cancel_date.date() if pd.notna(cancel_date) else 'NaT'}, DelayDays={delay_days}, CalcDelay={calc_delay}")

    # Only assert if no mismatches (for debugging, you might want to comment this out initially)
    assert len(mismatches) == 0, f"Delay calculation mismatch in {len(mismatches)} orders. Check debug output above."
    
    # Check 4: Platinum customer delays
    platinum_customers = df[df['customer_tier'] == 'Platinum']
    other_customers = df[df['customer_tier'] != 'Platinum']

    if len(platinum_customers) > 0 and len(other_customers) > 0:
        platinum_delays = platinum_customers['is_delayed'].mean()
        other_delays = other_customers['is_delayed'].mean()
        print(f"Platinum customer delay rate: {platinum_delays:.2%}")
        print(f"Other customer delay rate: {other_delays:.2%}")

        # Only assert if we have enough data
        if len(df) > 50:
            assert platinum_delays < other_delays * 0.7, "Platinum customers should have significantly fewer delays"
    else:
        print("Not enough data to compare Platinum vs other customer delays")
    
    # Final statistics
    print(f"\nDataset generated successfully!")
    print(f"Total rows: {len(df):,}")
    print(f"Date range: {df['order_date'].min()} to {df['order_date'].max()}")
    print(f"Unique customers: {df['customer_id'].nunique()}")
    print(f"Unique products: {df['product_id'].nunique()}")
    print(f"Overall delay rate: {df['is_delayed'].mean():.2%}")
    print(f"\nProduct distribution:")
    print(df['product_category'].value_counts(normalize=True))
    
    # Reorder columns for better readability
    column_order = [# Core order info
                    'order_id', 'order_date', 'customer_id', 'product_id', 'quantity',
        
                    # Product details
                    'product_category', 'vehicle_model', 'vehicle_class', 'configuration', 
                    'color', 'condition', 'age', 'miles_driven', 'base_price', 'current_price', 

                    # Financial
                    'order_value_usd', 'base_price_usd', 'options_value_usd', 'material_cost_usd', 
                    'labor_cost_usd', 'shipping_cost_usd', 'overhead_cost_usd', 'rush_order_premium_usd', 
                    'discount', 'rebate_usd', 'profit_margin_percent', 'payment_terms', 'payment_terms_days', 
                    'customer_credit_rating', 'currency', 

                    # Customer
                    'customer_name', 'customer_type', 'customer_tier', 'customer_region', 'fleet_size', 
                    'previous_orders_count', 'customer_satisfaction', 'sales_channel', 'customer_revenue_ytd', 

                    # Status and delivery
                    'order_status', 'cancel_reason', 'order_priority', 'requested_delivery_date', 
                    'planned_delivery_date', 'actual_delivery_date', 'cancellation_date', 
                    'cycle_time_days', 'on_promise_delivery', 'delivery_region', 
                    'delivery_distance', 'delivery_method', 

                    # Manufacturing
                    'production_plant', 'production_line', 'planned_production_start_date', 
                    'actual_production_start_date', 'planned_production_end_date', 'actual_production_end_date', 
                    'build_complexity', 'production_capacity_used', 'capacity_available_percent', 
                    'labor_hours_required', 'production_sequence', 'competing_orders_count', 
        
                    # Engineering
                    'engineering_status', 'engineering_approval_date', 'custom_engineering_required', 
                    'technical_complexity_score', 'design_changes_count', 

                    # Supply chain
                    'inventory_shortage_count', 'parts_availability', 'backorder_parts_count', 
                    'longest_lead_time_days', 'component_sourcing_status', 'supply_risk_score', 
                    'key_supplier_issues', 'supply_chain_status', 

                    # Constraints and delays
                    'is_delayed', 'delay_days', 'primary_delay_category', 'delay_reason', 
                    'secondary_delay_category', 'revenue_impact_usd', 'delay_impact_severity', 
                    'constraint_resolved', 'resolution_days', 'escalation_level', 
                    'previous_constraints_count', 'constraint_start_date', 'expected_resolution_date', 
                    'days_until_requirement', 

                    # Quality
                    'quality_check_status', 'quality_score', 'rework_required', 

                    # Temporal
                    'order_year', 'order_month', 'order_quarter', 'season', 'is_holiday_period', 'demand_factor', 

                    # Promotional
                    'promo_available', 'promo_category', 'promo_value_discount', 'promo_value_rebate_usd', 'promo_applied',
        
                    # Additional costs
                    'storage_holding_cost_usd', 'late_delivery_penalty_usd'
                    ]
    
    df = df[column_order]    
    return df

In [11]:
# Generate the dataset
if __name__ == "__main__":
    # Constants
    START_DATE = datetime(2023, 6, 1)
    END_DATE = datetime(2025, 5, 31)
    TARGET_ROWS = 60000
    
    df = generate_dataset(START_DATE, END_DATE, TARGET_ROWS)
    filename = 'Commercial_Vehicle_Orders_Synthetic_Dataset.csv'
    df.to_csv(filename, index=False)
    
    df

Starting dataset generation...
Processing June 2023...
Processing July 2023...
Processing August 2023...
Processing September 2023...
Processing October 2023...
Processing November 2023...
Processing December 2023...
Processing January 2024...
Processing February 2024...
Processing March 2024...
Processing April 2024...
Processing May 2024...
Processing June 2024...
Processing July 2024...
Processing August 2024...
Processing September 2024...
Processing October 2024...
Processing November 2024...
Processing December 2024...
Processing January 2025...
Processing February 2025...
Processing March 2025...
Processing April 2025...
Processing May 2025...

Generated 60157 order line items

Cleaning up data values...
Data cleanup completed!
- Categorical columns: None/NaN → 'N/A'
- Numerical columns: Kept as NaN where appropriate
- Date columns: None → NaT

DataFrame shape: (60157, 101)

Running data quality checks...
Platinum customer delay rate: 16.25%
Other customer delay rate: 26.93%

Da

In [12]:
df

,order_id,order_date,customer_id,product_id,quantity,product_category,vehicle_model,vehicle_class,configuration,color,...,season,is_holiday_period,demand_factor,promo_available,promo_category,promo_value_discount,promo_value_rebate_usd,promo_applied,storage_holding_cost_usd,late_delivery_penalty_usd
0,CV-2023-045821,2023-06-01,CUST-10012,BS26656,5,Bus,Transit Bus 40ft,City Bus,"Low Floor, Diesel, 280HP",Green,...,Summer,False,1.3,False,N/A,0.00,0,N/A,0.000000,0.0
1,CV-2023-045821,2023-06-01,CUST-10012,BS22492,5,Bus,Transit Bus 40ft,City Bus,"Low Floor, Diesel, 280HP",Black,...,Summer,False,1.3,False,N/A,0.00,0,N/A,0.000000,0.0
2,CV-2023-045822,2023-06-01,CUST-10012,TR79562,1,Truck,eCascadia,Class 8 Truck,"Electric, Day Cab, 470HP",Green,...,Summer,False,1.0,True,Volume,1.72,2000,True,2205.584597,0.0
3,CV-2023-045823,2023-06-01,CUST-10017,BS27669,5,Bus,School Bus C2,School Bus,"Type C, Diesel, 240HP",Red,...,Summer,False,1.3,False,N/A,0.00,0,N/A,790.343818,0.0
4,CV-2023-045824,2023-06-01,CUST-10017,BS25918,20,Bus,Transit Bus 60ft,City Bus,"Articulated, Hybrid, 330HP",Black,...,Summer,False,1.3,False,N/A,0.00,0,N/A,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60152,CV-2025-083337,2025-05-30,CUST-10049,PT66084,1,Powertrain,DD15 Engine,Heavy Duty Engine,"14.8L, 505HP, EPA 2021",Blue,...,Spring,False,1.2,False,N/A,0.00,0,N/A,0.000000,0.0
60153,CV-2025-083337,2025-05-30,CUST-10049,TR78241,1,Truck,LT Series,Class 7 Truck,"Extended Cab, Auto Trans, 360HP",White,...,Spring,False,1.2,True,Federal Holiday,3.55,0,False,0.000000,0.0
60154,CV-2025-083337,2025-05-30,CUST-10049,TR78739,1,Truck,Cascadia DD15,Class 8 Truck,"Sleeper Cab, Auto Trans, 455HP",Black,...,Spring,False,1.2,False,N/A,0.00,0,N/A,0.000000,0.0
60155,CV-2025-083338,2025-05-30,CUST-10014,BS22599,5,Bus,Transit Bus 40ft,City Bus,"Low Floor, Diesel, 280HP",Black,...,Spring,False,1.2,False,N/A,0.00,0,N/A,0.000000,0.0


In [13]:
df[['order_date', 'requested_delivery_date', 'planned_delivery_date', 'actual_delivery_date', 'is_delayed', 
    'delay_days', 'planned_production_start_date', 'actual_production_start_date', 'planned_production_end_date', 
    'actual_production_end_date', 'engineering_approval_date', 'constraint_start_date', 
    'expected_resolution_date', 'order_status']].sort_values(by='actual_production_start_date', ascending=False)

,order_date,requested_delivery_date,planned_delivery_date,actual_delivery_date,is_delayed,delay_days,planned_production_start_date,actual_production_start_date,planned_production_end_date,actual_production_end_date,engineering_approval_date,constraint_start_date,expected_resolution_date,order_status
59372,2025-05-20,2025-10-12,2025-09-13,NaT,False,0,2025-05-30,2025-05-31,2025-08-24,NaT,2025-05-25,NaT,NaT,In_Production
59359,2025-05-20,2025-09-02,2025-08-10,NaT,False,0,2025-05-31,2025-05-31,2025-08-04,NaT,2025-05-26,NaT,NaT,In_Production
59349,2025-05-20,2025-10-23,2025-09-23,NaT,False,0,2025-05-31,2025-05-31,2025-09-04,NaT,2025-05-25,NaT,NaT,In_Production
59195,2025-05-19,2025-11-03,2025-09-11,NaT,False,0,2025-05-29,2025-05-31,2025-09-01,NaT,2025-05-22,NaT,NaT,In_Production
59201,2025-05-19,2025-06-26,2025-07-05,NaT,False,0,2025-05-31,2025-05-31,2025-06-21,NaT,2025-05-25,NaT,NaT,In_Production
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60152,2025-05-30,2025-07-21,2025-07-25,NaT,False,0,2025-06-07,NaT,2025-07-06,NaT,2025-06-04,NaT,NaT,Pending
60153,2025-05-30,2025-10-14,2025-09-06,NaT,False,0,2025-06-13,NaT,2025-08-28,NaT,2025-06-01,NaT,NaT,Pending
60154,2025-05-30,2025-10-01,2025-09-13,NaT,False,0,2025-06-04,NaT,2025-08-19,NaT,2025-06-03,NaT,NaT,Pending
60155,2025-05-30,2025-12-07,2025-10-07,NaT,False,0,2025-06-06,NaT,2025-09-26,NaT,2025-06-08,NaT,NaT,Pending


In [14]:
df[['requested_delivery_date', 'actual_delivery_date', 
    'on_promise_delivery', 'is_delayed', 
    'delay_reason', 'primary_delay_category', 'secondary_delay_category', 
    'escalation_level', 'constraint_resolved', 
    'delay_days', 'revenue_impact_usd', 'delay_impact_severity', 
    'key_supplier_issues', 'quality_check_status', 'rework_required', 'promo_category']]

,requested_delivery_date,actual_delivery_date,on_promise_delivery,is_delayed,delay_reason,primary_delay_category,secondary_delay_category,escalation_level,constraint_resolved,delay_days,revenue_impact_usd,delay_impact_severity,key_supplier_issues,quality_check_status,rework_required,promo_category
0,2023-12-01,2023-10-03,True,False,N/A,N/A,N/A,N/A,N/A,0,0.0,N/A,N/A,Passed,False,N/A
1,2023-11-15,2023-10-09,True,False,N/A,N/A,N/A,N/A,N/A,0,0.0,N/A,N/A,Passed,False,N/A
2,2023-07-03,2023-07-21,False,True,Glass shortage,Parts/Material,N/A,Manager,True,18,0.0,High,N/A,Passed,False,Volume
3,2023-07-28,2023-08-01,False,True,Steel allocation,Parts/Material,Customer,Supervisor,True,4,0.0,Medium,N/A,Failed,True,N/A
4,2023-11-10,2023-09-26,True,False,N/A,N/A,N/A,N/A,N/A,0,0.0,N/A,N/A,Passed,False,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60152,2025-07-21,NaT,N/A,False,N/A,N/A,N/A,N/A,N/A,0,0.0,N/A,N/A,N/A,N/A,N/A
60153,2025-10-14,NaT,N/A,False,N/A,N/A,N/A,N/A,N/A,0,0.0,N/A,N/A,N/A,N/A,Federal Holiday
60154,2025-10-01,NaT,N/A,False,N/A,N/A,N/A,N/A,N/A,0,0.0,N/A,N/A,N/A,N/A,N/A
60155,2025-12-07,NaT,N/A,False,N/A,N/A,N/A,N/A,N/A,0,0.0,N/A,N/A,N/A,N/A,N/A


In [15]:
for column in ['delay_reason', 'primary_delay_category', 'secondary_delay_category', 'escalation_level', 
               'constraint_resolved', 'key_supplier_issues', 'quality_check_status', 'rework_required', 
               'delay_impact_severity', 'promo_category', 'promo_applied']: 
    print (column + ":", df[column].unique())
    print ()

delay_reason: ['N/A' 'Glass shortage' 'Steel allocation' 'Behind schedule'
 'Tire shortage' 'Wiring harness delay' 'Custom feature development'
 'Safety standard update' 'Battery cells unavailable'
 'Design revision required' 'Quality hold' 'Semiconductor shortage'
 'Compliance update needed' 'Capacity constraint' 'Tool breakdown'
 'Assembly error' 'Labor shortage' 'Customer spec changes'
 'Rework required' 'Previous order overrun' 'Contract negotiation'
 'Supplier quality issue' 'Line maintenance' 'Payment pending'
 'Paint defect' 'Failed inspection' 'EPA certification'
 'Specification unclear' 'Delivery postponed' 'Import permit'
 'Fleet readiness' 'Safety compliance' 'DOT approval']

primary_delay_category: ['N/A' 'Parts/Material' 'Production' 'Engineering' 'Quality' 'Customer'
 'Regulatory']

secondary_delay_category: ['N/A' 'Customer' 'Production' 'Regulatory' 'Engineering' 'Parts/Material'
 'Quality']

escalation_level: ['N/A' 'Manager' 'Supervisor' 'Director']

constraint_resolv

In [30]:
for column in df: 
    if 'datetime' not in str(df[column].dtype): 
        if 'object' not in str(df[column].dtype): 
            print (column, df[column].dtype)
            print (df[column].value_counts)

quantity int64
<bound method IndexOpsMixin.value_counts of 0         5
1         5
2         1
3         5
4        20
         ..
60152     1
60153     1
60154     1
60155     5
60156     1
Name: quantity, Length: 60157, dtype: int64>
age float64
<bound method IndexOpsMixin.value_counts of 0        0.0
1        0.0
2        4.3
3        0.7
4        0.0
        ... 
60152    0.0
60153    0.0
60154    2.6
60155    0.0
60156    0.0
Name: age, Length: 60157, dtype: float64>
miles_driven int64
<bound method IndexOpsMixin.value_counts of 0            22
1            43
2        154280
3         19276
4            47
          ...  
60152         7
60153        43
60154     82437
60155         7
60156        41
Name: miles_driven, Length: 60157, dtype: int64>
base_price int64
<bound method IndexOpsMixin.value_counts of 0        285000
1        285000
2        285000
3        155000
4        425000
          ...  
60152     42000
60153    125000
60154    145000
60155    285000
60156    16500

In [17]:
df[df['order_status']=='Canceled'][['requested_delivery_date', 'actual_delivery_date', 'on_promise_delivery', 
                                    'cancellation_date', 'cancel_reason', 'is_delayed', 'delay_days', ]]

,requested_delivery_date,actual_delivery_date,on_promise_delivery,cancellation_date,cancel_reason,is_delayed,delay_days
8,2023-11-29,NaT,N/A,2023-11-14,Other,False,0
28,2023-12-02,NaT,N/A,2023-10-14,Other,False,0
31,2023-09-10,NaT,N/A,2023-07-12,Customer Reasons,False,0
44,2023-10-20,NaT,N/A,2023-07-20,Customer Reasons,False,0
55,2024-01-05,NaT,N/A,2024-02-21,Delay,True,47
...,...,...,...,...,...,...,...
59431,2025-10-19,NaT,N/A,2025-06-04,Customer Reasons,False,0
59448,2025-09-23,NaT,N/A,2025-06-05,Customer Reasons,False,0
59456,2025-11-05,NaT,N/A,2025-05-29,Customer Reasons,False,0
59492,2025-11-28,NaT,N/A,2025-06-09,Customer Reasons,False,0


In [18]:
ls = []
for col in df.columns: 
    if 'delay' in col: 
        ls.append(col)
        
print (ls)

['is_delayed', 'delay_days', 'primary_delay_category', 'delay_reason', 'secondary_delay_category', 'delay_impact_severity']


In [19]:
df[df['is_delayed'] == True][['requested_delivery_date', 'actual_delivery_date', 'is_delayed', 'delay_days', 
                              'primary_delay_category', 'delay_reason', 'secondary_delay_category', 
                              'delay_impact_severity', 'order_value_usd', 'revenue_impact_usd', 'customer_tier']]

,requested_delivery_date,actual_delivery_date,is_delayed,delay_days,primary_delay_category,delay_reason,secondary_delay_category,delay_impact_severity,order_value_usd,revenue_impact_usd,customer_tier
2,2023-07-03,2023-07-21,True,18,Parts/Material,Glass shortage,N/A,High,1.070026e+05,0.00,Silver
3,2023-07-28,2023-08-01,True,4,Parts/Material,Steel allocation,Customer,Medium,8.066087e+05,0.00,Gold
10,2023-08-27,2023-09-16,True,20,Production,Behind schedule,N/A,Medium,3.993659e+06,0.00,Gold
12,2023-07-20,2023-08-01,True,12,Parts/Material,Glass shortage,N/A,High,9.382341e+04,2945.99,Gold
14,2023-09-16,2023-09-24,True,8,Production,Behind schedule,N/A,Low,1.556280e+06,0.00,Gold
...,...,...,...,...,...,...,...,...,...,...,...
58600,2025-05-20,NaT,True,11,Production,Behind schedule,N/A,Low,2.469890e+04,0.00,Silver
58712,2025-05-23,NaT,True,8,Parts/Material,Steel allocation,N/A,High,2.594397e+04,0.00,Bronze
58760,2025-05-28,NaT,True,3,Production,Behind schedule,N/A,Low,7.295536e+04,0.00,Silver
58893,2025-05-26,NaT,True,5,Production,Behind schedule,N/A,Low,1.933436e+04,0.00,Bronze


In [20]:
df[(df['is_delayed'] == True) | (df['order_status']=='Canceled')]

,order_id,order_date,customer_id,product_id,quantity,product_category,vehicle_model,vehicle_class,configuration,color,...,season,is_holiday_period,demand_factor,promo_available,promo_category,promo_value_discount,promo_value_rebate_usd,promo_applied,storage_holding_cost_usd,late_delivery_penalty_usd
2,CV-2023-045822,2023-06-01,CUST-10012,TR79562,1,Truck,eCascadia,Class 8 Truck,"Electric, Day Cab, 470HP",Green,...,Summer,False,1.0,True,Volume,1.72,2000,True,2205.584597,0.0
3,CV-2023-045823,2023-06-01,CUST-10017,BS27669,5,Bus,School Bus C2,School Bus,"Type C, Diesel, 240HP",Red,...,Summer,False,1.3,False,N/A,0.00,0,N/A,790.343818,0.0
8,CV-2023-045826,2023-06-01,CUST-10012,BS27577,10,Bus,Electric Transit 35ft,City Bus,"Electric, Low Floor, 350HP",Silver,...,Summer,False,1.3,False,N/A,0.00,0,N/A,0.000000,0.0
10,CV-2023-045828,2023-06-01,CUST-10017,BS26133,10,Bus,Electric Transit 35ft,City Bus,"Electric, Low Floor, 350HP",Blue,...,Summer,False,1.3,False,N/A,0.00,0,N/A,2739.378531,0.0
12,CV-2023-045829,2023-06-01,CUST-10017,PT61461,2,Powertrain,X15 Engine,Heavy Duty Engine,"15L, 605HP, EPA 2021",White,...,Summer,False,1.0,True,Government,2.78,0,True,698.447714,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59431,CV-2025-082893,2025-05-21,CUST-10042,TR74380,1,Truck,M2 106,Class 6 Truck,"Crew Cab, Auto Trans, 300HP",Green,...,Spring,False,1.2,False,N/A,0.00,0,N/A,0.000000,0.0
59448,CV-2025-082906,2025-05-21,CUST-10062,TR77549,1,Truck,Western Star 57X,Class 8 Truck,"Sleeper Cab, Manual Trans, 505HP",Red,...,Spring,False,1.2,False,N/A,0.00,0,N/A,0.000000,0.0
59456,CV-2025-082909,2025-05-21,CUST-10065,PT60902,1,Powertrain,Allison 3000,Automatic Trans,"6-Speed, Heavy Duty",Red,...,Spring,False,1.2,True,Trade-in,0.00,10000,False,0.000000,0.0
59492,CV-2025-082932,2025-05-21,CUST-10032,TR74257,1,Truck,Western Star 57X,Class 8 Truck,"Sleeper Cab, Manual Trans, 505HP",Blue,...,Spring,False,1.2,False,N/A,0.00,0,N/A,0.000000,0.0


In [21]:
df[df['is_delayed'] == True]['revenue_impact_usd'].unique()

array([    0.        ,  2945.99      , 74721.68693381, ...,
       14729.95961032,  1055.2       ,  1213.9       ])

In [32]:
df[(df['order_status']=='Canceled')][['cancel_reason', 'order_value_usd', 'revenue_impact_usd']]

,cancel_reason,order_value_usd,revenue_impact_usd
8,Other,4.004406e+06,7.116244e+06
28,Other,2.856693e+06,5.030953e+06
31,Customer Reasons,5.018095e+05,6.559817e+05
44,Customer Reasons,1.278914e+05,2.024165e+05
55,Delay,4.309401e+04,7.472169e+04
...,...,...,...
59431,Customer Reasons,1.131873e+05,1.197630e+05
59448,Customer Reasons,1.725552e+05,2.417048e+05
59456,Customer Reasons,1.351469e+04,1.434712e+04
59492,Customer Reasons,1.672422e+05,1.824913e+05


In [23]:
df[df['is_delayed'] == True]['delay_reason'].unique()

array(['Glass shortage', 'Steel allocation', 'Behind schedule',
       'Tire shortage', 'Wiring harness delay',
       'Custom feature development', 'Safety standard update',
       'Battery cells unavailable', 'Design revision required',
       'Quality hold', 'Semiconductor shortage',
       'Compliance update needed', 'Capacity constraint',
       'Tool breakdown', 'Assembly error', 'Labor shortage',
       'Customer spec changes', 'Rework required',
       'Previous order overrun', 'Contract negotiation',
       'Supplier quality issue', 'Line maintenance', 'Payment pending',
       'Paint defect', 'Failed inspection', 'EPA certification',
       'Specification unclear', 'Delivery postponed', 'Import permit',
       'Fleet readiness', 'Safety compliance', 'DOT approval'],
      dtype=object)

In [33]:
df[df['primary_delay_category'] == 'Customer']

,order_id,order_date,customer_id,product_id,quantity,product_category,vehicle_model,vehicle_class,configuration,color,...,season,is_holiday_period,demand_factor,promo_available,promo_category,promo_value_discount,promo_value_rebate_usd,promo_applied,storage_holding_cost_usd,late_delivery_penalty_usd
1314,CV-2023-046624,2023-06-15,CUST-10061,PT64165,1,Powertrain,X15 Engine,Heavy Duty Engine,"15L, 605HP, EPA 2021",Green,...,Summer,False,1.0,False,N/A,0.0,0,N/A,2618.099629,0.0
1588,CV-2023-046797,2023-06-19,CUST-10058,PT64000,1,Powertrain,Allison 3000,Automatic Trans,"6-Speed, Heavy Duty",Black,...,Summer,False,1.0,False,N/A,0.0,0,N/A,2774.941113,0.0
2301,CV-2023-047230,2023-06-28,CUST-10063,TR78661,1,Truck,Cascadia DD13,Class 8 Truck,"Day Cab, Manual Trans, 410HP",Silver,...,Summer,False,1.0,False,N/A,0.0,0,N/A,8142.610726,0.0
3353,CV-2023-047888,2023-07-10,CUST-10028,BS28518,1,Bus,Transit Bus 40ft,City Bus,"Low Floor, Diesel, 280HP",Yellow,...,Summer,True,1.3,False,N/A,0.0,0,N/A,2124.109911,0.0
3800,CV-2023-048177,2023-07-13,CUST-10054,PT68688,1,Powertrain,Allison 3000,Automatic Trans,"6-Speed, Heavy Duty",Black,...,Summer,True,1.0,False,N/A,0.0,0,N/A,5920.813571,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49024,CV-2025-076358,2025-01-10,CUST-10056,PT66675,1,Powertrain,Allison 3000,Automatic Trans,"6-Speed, Heavy Duty",Yellow,...,Winter,True,0.8,False,N/A,0.0,0,N/A,3315.310312,0.0
49548,CV-2025-076687,2025-01-16,CUST-10043,PT68926,1,Powertrain,X12 Engine,Heavy Duty Engine,"12L, 500HP, EPA 2021",Silver,...,Winter,True,0.8,False,N/A,0.0,0,N/A,2972.095186,0.0
52421,CV-2025-078466,2025-02-21,CUST-10059,PT68755,1,Powertrain,DD15 Engine,Heavy Duty Engine,"14.8L, 505HP, EPA 2021",Black,...,Winter,False,0.8,True,Trade-in,0.0,10000,False,1372.991078,0.0
53202,CV-2025-078968,2025-03-04,CUST-10043,PT67768,1,Powertrain,X12 Engine,Heavy Duty Engine,"12L, 500HP, EPA 2021",Black,...,Spring,False,1.2,False,N/A,0.0,0,N/A,2195.370709,0.0


In [24]:
df[(df['is_delayed'] == True)]['primary_delay_category'].unique()

array(['Parts/Material', 'Production', 'Engineering', 'Quality',
       'Customer', 'Regulatory'], dtype=object)

In [25]:
df[(df['is_delayed'] == True)]['delay_impact_severity'].unique()

array(['High', 'Medium', 'Low', 'Critical'], dtype=object)

In [26]:
df[(df['is_delayed']==True) & (df['order_status']=='Canceled') & (df['cancel_reason']!='Delay')][['requested_delivery_date', 'cancellation_date', 'order_status', 'cancel_reason', 'delay_reason', 'primary_delay_category', 'delay_impact_severity']]

,requested_delivery_date,cancellation_date,order_status,cancel_reason,delay_reason,primary_delay_category,delay_impact_severity


In [27]:
df['is_delayed'].unique()

array([False,  True])

In [28]:
df[(df['is_delayed'] == True) & (df['primary_delay_category']=='N/A')]

,order_id,order_date,customer_id,product_id,quantity,product_category,vehicle_model,vehicle_class,configuration,color,...,season,is_holiday_period,demand_factor,promo_available,promo_category,promo_value_discount,promo_value_rebate_usd,promo_applied,storage_holding_cost_usd,late_delivery_penalty_usd
